# Monopoly - probability of hitting a particular space after N cycles


_The game of monopoly is played on a square board with 10 spaces per side (so 40 in total). A player starts at the space marked "go" and on each turn rolls two sided dice, advancing forward a number of spaces equal to the sum of the two numbers that lands face up. (if a player rolls a sum that would normally cause them to advance past the 40th square, they will instead loop around, so if a player landed on a hypothetical 44th square, they will instead go to the 4th square on the board). What is the probability that a player after they "loop around" the board 9 times, lands on the space "Boardwalk" (the 40th square) before they "loop around" the board an eleventh time?_

It's easy to calculate probability of moving 1, 2, ..., 12 spaces forward. However, it's not quite obvious how to calculate the probability of hitting space 40 after 9 trips around the board. We could rephrase it as - starting from 0, keep throwing two dice and adding the score. What is the probability of the sum being exactly 400 at some point? 

# Quick check - Monte Carlo

In [7]:
import numpy


results = []
for i in range(5000):
    total_sum = 0
    while (total_sum < 400):
        dice1 = numpy.random.choice([1, 2, 3, 4, 5, 6])
        dice2 = numpy.random.choice([1, 2, 3, 4, 5, 6])
        total_sum += dice1 + dice2
    if total_sum == 400:
        results.append(1)
    else:
        results.append(0)

print(numpy.mean(results))

0.1416


This number seem to be close $1 / E(d_1 + d_2) = 1 / 7 = 0.142857143$ - how can we get there?

# Markov chain with absorbing final states

A more accurate way to model this is by using a Markov chain. Let's have a chain with 401 states. States from 0 to 399 model progression from some state of the board to the next 12 states. Probabilities are as follows:
```
Roll a… 	Probability
1             0
2 	        1/36 (2.778%)
3 	        2/36 (5.556%)
4 	        3/36 (8.333%)
5 	        4/36 (11.111%)
6 	        5/36 (13.889%)
7 	        6/36 (16.667%)
8 	        5/36 (13.889%)
9 	        4/36 (11.111%)
10 	       3/36 (8.333%)
11 	       2/36 (5.556%)
12 	       1/36 (2.778%)
```

With this in hand we build the transition matrix as follows:

In [19]:
# two zeros because the first element is P(staying in the same state)
P = numpy.r_[[0, 0, 1./36, 2/36, 3/36, 4/36, 5/36, 6/36, 5/36, 4/36, 3/36, 2/36, 1/36], numpy.zeros(388)]
T = []

for i in range(0, 401):
    T.append(numpy.roll(P, i))

T = numpy.array(T)

# now special arrangements for last rows
# we make states 400 and 401 absorbing - they have P(staying in the state) = 1
T[-2] = numpy.r_[numpy.zeros(399), [1, 0]]
T[-1] = numpy.r_[numpy.zeros(400), 1]

# when we do roll, non-zero probabilities start wrapping over back to states 0,1,etc - but we instead want them
# to accummulate in 401 (modelling "overshoot") 
for i in range(-12, -2):
    T[i, -1] += sum(T[i, :12])
    T[i, :12] = 0

Now to calculate probability of moving from state $i$ to state $j$ in $n$ steps, we just need to calculate $T^n$ and look at $T_{ij}$. Let's check this - after 5 steps we can be in state 5 * 12 = 60 (with a very low probability = $(1/36)^{5}$) but no further

In [22]:
T5 = numpy.linalg.matrix_power(T, 5)
print(T5[0, 60],  T5[0,61])

1.6538171687920198e-08 0.0


Now a twist - because we have modelled states 400 and 401 as "absorbing states", we can just raise the matrix to the power of at least 200, so that we definitely reach either 400 (200 * 2) or more.

In [23]:
T200 = numpy.linalg.matrix_power(T, 210)

In [25]:
T200[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

As expected probabilities in all spaces apart from 400 and 401 are zero. Note that $P(400) = 1/7$ as predicted by the monte carlo test. 

# Analytical proof?

Markov model above in effect does the explicit calculation - it adds up probabilities along all paths of all possible lengths. Therefore, it constitutes a computational proof, however having a simpler one would be nicer. Why is it $1/E(d_1 + d_2)$ ?